In [1]:
%%html
<style>
.h1_cell, .just_text {
    box-sizing: border-box;
    padding-top:5px;
    padding-bottom:5px;
    font-family: "Times New Roman", Georgia, Serif;
    font-size: 125%;
    line-height: 22px; /* 5px +12px + 5px */
    text-indent: 25px;
    background-color: #fbfbea;
    padding: 10px;
}

hr { 
    display: block;
    margin-top: 0.5em;
    margin-bottom: 0.5em;
    margin-left: auto;
    margin-right: auto;
    border-style: inset;
    border-width: 2px;
}
</style>

<h1>
<center>
Module 7: K-Means and Empties
</center>
</h1>
<div class=h1_cell>

You will be working with the loan table again.

</div>

In [108]:
import pandas as pd
import os

week = 5

home_path =  os.path.expanduser('~')

file_path = '/Desktop/CIS399/'

file_name = 'loan_wrangled_w'+str(week)+'.csv'

loan_table = pd.read_csv(home_path + file_path + file_name)

In [109]:
os.chdir(home_path + file_path + 'datascience_1/')
!git pull

Already up-to-date.


In [110]:
#load the lbirary from content this week

import sys
sys.path.append(home_path + '...')
from week7 import *
%who function

accuracy	 build_pred	 build_tree_iter	 caser	 closest_centroid	 compute_centroid_labels	 compute_mean	 compute_prediction	 compute_training	 
euclidean_distance	 f1	 find_best_splitter	 forest_builder	 generate_table	 gig	 gini	 informedness	 initialize_centroids	 
k_fold	 k_means	 kmeans_fill	 phase_1	 phase_2	 predictor_case	 probabilities	 row_to_vect	 seeder	 
tree_predictor	 vote_taker	 


In [56]:
pd.set_option('display.max_columns', None)
loan_table.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,no_lam,filled_lam,pa_Rural,pa_Semiurban,pa_Urban,pa_nan,bin_lam,bin_Low,bin_Average,bin_High,ch_bad,ch_good,ch_nan,apin_binned,apin_Low,apin_Average,apin_High,apin_nan,dep_0,dep_1,dep_2,dep_3+
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,1,1,146.412162,0,0,1,0,Low,1,0,0,0,1,0,Low,1,0,0,0,1,0,0,0
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,0,0,128.000000,1,0,0,0,Low,1,0,0,0,1,0,Low,1,0,0,0,0,1,0,0
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,1,0,66.000000,0,0,1,0,Low,1,0,0,0,1,0,Low,1,0,0,0,1,0,0,0
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,1,0,120.000000,0,0,1,0,Low,1,0,0,0,1,0,Low,1,0,0,0,1,0,0,0
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,1,0,141.000000,0,0,1,0,Low,1,0,0,0,1,0,Low,1,0,0,0,1,0,0,0


<hr>
<h1>
Look for column with largest number of empties
</h1>
<p>
<div class=h1_cell>
<p>
I'll use what we have learned in past modules to find the column with the largest number of empties. I am going to set the column name to `focus_column`.
</div>

In [57]:
loan_table.describe()  # can use count to see columns that are missing values

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,no_lam,filled_lam,pa_Rural,pa_Semiurban,pa_Urban,pa_nan,bin_Low,bin_Average,bin_High,ch_bad,ch_good,ch_nan,apin_Low,apin_Average,apin_High,apin_nan,dep_0,dep_1,dep_2,dep_3+
count,614.000000,614.000000,592.000000,600.00000,564.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.0,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.0,614.000000,614.000000,614.000000,614.000000
mean,5403.459283,1621.245798,146.412162,342.00000,0.842199,0.687296,0.035831,146.412162,0.291531,0.379479,0.328990,0.0,0.905537,0.074919,0.019544,0.144951,0.773616,0.081433,0.988599,0.008143,0.003257,0.0,0.561889,0.166124,0.164495,0.083062
std,6109.041673,2926.248369,85.587325,65.12041,0.364878,0.463973,0.186019,84.037468,0.454838,0.485653,0.470229,0.0,0.292710,0.263475,0.138540,0.352339,0.418832,0.273722,0.106250,0.089945,0.057026,0.0,0.496559,0.372495,0.371027,0.276201
min,150.000000,0.000000,9.000000,12.00000,0.000000,0.000000,0.000000,9.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
25%,2877.500000,0.000000,100.000000,360.00000,1.000000,0.000000,0.000000,100.250000,0.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
50%,3812.500000,1188.500000,128.000000,360.00000,1.000000,1.000000,0.000000,129.000000,0.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000
75%,5795.000000,2297.250000,168.000000,360.00000,1.000000,1.000000,0.000000,164.750000,1.000000,1.000000,1.000000,0.0,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000
max,81000.000000,41667.000000,700.000000,480.00000,1.000000,1.000000,1.000000,700.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000


In [58]:
loan_table['LoanAmount'].isnull().sum()  # looks like LoanAmount is a good target

22

In [59]:
focus_column =  'LoanAmount' #column with largest number of empties

<hr>
<h2>
Use these as the features you will build each point from.
</h2>
<p>
<div class=h1_cell>
<p>
So you should have each point be a tuple of 5 values.
</div>

In [60]:
features_used = [
        #Income
       'ApplicantIncome', 'CoapplicantIncome',
        #Property_Area
        'pa_Rural', 'pa_Semiurban','pa_Urban'
        ]

<hr>
<h2>
Crank up k_means
</h2>
<p>
<div class=h1_cell>
<p>
Let's get the final 5 centroid values.
</div>

In [61]:
k = 5

In [62]:
final_centroids = k_means(loan_table, features_used, k, {'n':100, 'threshold':0.0})

starting 1
starting 2
starting 3
starting 4
starting 5
starting 6
starting 7
starting 8
starting 9
starting 10
starting 11
starting 12
starting 13
starting 14
starting 15
starting 16
starting 17
starting 18
starting 19
starting 20
starting 21
starting 22
starting 23
starting 24
starting 25
starting 26
starting 27
starting 28
starting 29
starting 30
done


In [63]:
for i in range(k):
    print(final_centroids[i]['centroid'])

[1588.0, 28876.0, 0.0, 0.25, 0.75]
[49544.42857142857, 678.5714285714286, 0.14285714285714285, 0.5714285714285714, 0.2857142857142857]
[13406.360655737704, 651.4754098360655, 0.2786885245901639, 0.32786885245901637, 0.39344262295081966]
[4648.5161290322585, 307.93232975211464, 0.2867383512544803, 0.3942652329749104, 0.31899641577060933]
[3231.319391634981, 2849.9536121209126, 0.30798479087452474, 0.3726235741444867, 0.3193916349809886]


In [64]:
for i in range(k):
    print(len(final_centroids[i]['cluster']))

4
7
61
279
263


<hr>
<h2>
Turning it over to you
</h2>
<p>
<div class=h1_cell>
<p>
You now have your 5 centroids. Your goal is to create a new column `kmeans_LoanAmount` that fills empties with value attached to closest cluster. I'll break it into steops.
</div>

<h2>
Step 1. Create a sub-table that has values for LoanAmount
</h2>
<p>
<div class=h1_cell>
<p>
Normal use of pandas.
</div>

In [65]:
import numpy

lam_table = loan_table.dropna(subset=['LoanAmount'])
#print(lam_table.head())
print(len(lam_table))




592


<h2>
Step 2. Label each centroid
</h2>
<p>
<div class=h1_cell>
<p>
Go through all the rows in lam_table. For each row, find its closest centroid. Add the LoanAmount to that centroid. When finished, add the amounts for each centroid and divide by length to get the mean. Store that on the centroid as `mean_label`.
<p>
I started my function out as below. You do not have to follow this.
</div>

In [66]:
def compute_centroid_labels(centroids, focus_table, focus_column, features, k):


    for i in range(k):
        centroids[i]['means_cluster'] = [] 
        #print(final_centroids[i]['means_cluster'], i)

    for i in range(len(focus_table)):
        row = focus_table.iloc[i] #go through each row 
        vrow = row_to_vect(row, features) #for calculating which is closest centroid
        idx, closest = closest_centroid(centroids, vrow, k) #Get's closest centroid and index (cluster value)
        centroids[idx]['means_cluster'].append(row[focus_column]) #appen the centroid + LoanAmount value to each list 
       #print(row['LoanAmount'], idx)
        #print(idx)
        #closest_centroid(final_centroids, row, k)


    mean_labels = []
    for i in range(len(centroids)):
        #print(sum(final_centroids[i]['means_cluster']) / len(final_centroids[i]['means_cluster'])) #return the sum of each list / length of that list
        means_cluster = numpy.mean(centroids[i]['means_cluster'])
        mean_labels.append(means_cluster)

    for i in range(k):
        centroids[i]['mean_label'] = mean_labels[i]

    return centroids


In [67]:
centroids = compute_centroid_labels(final_centroids, lam_table, 'LoanAmount', features_used, k)

print(centroids[1]['mean_label'])

383.142857143


In [85]:
for i in range(k):
    print(centroids[i]['mean_label'])

195.5
383.142857143
251.379310345
122.322222222
140.731225296


<h2>
Step 3. Create the kmeans_LoanAmount column
</h2>
<p>
<div class=h1_cell>
<p>
Go through all the rows in loan_table. For each row, if it has a value for LoanAmount, just copy it. If it does not have a value then find the nearest centroid and use the `mean_label` on that centroid. Here is the start of my function:
</div>

In [112]:
import numpy as np

# new_table = loan_table

# for i in range(len(new_table)):
#     row = new_table.iloc[i] #go through each row 
#     if np.isnan(row["LoanAmount"]) == True:
#         vrow = row_to_vect(row, features_used) #for calculating which is closest centroid
#         idx, closest = closest_centroid(centroids, vrow, k) #Get's closest centroid and index (cluster value)
#         value = centroids[idx]['mean_label']
#         print(new_table.loc[i]["LoanAmount"])
#         new_table.loc[i, "LoanAmount"] = value # Fighting to get rid of that stupid chaining 
        


def kmeans_fill(centroids, full_table, features, focus_column, k):
    new_table = full_table 

    for i in range(len(new_table)):
        row = new_table.iloc[i]
        if np.isnan(row[focus_column]) == True:
            vrow = row_to_vect(row, features)
            idx, closest = closest_centroid(centroids, vrow, k)
            value = centroids[idx]['mean_label']
            new_table.loc[i, focus_column] = value
    return new_table


In [113]:

loan_table = kmeans_fill(centroids, loan_table, features_used, focus_column, k)
loan_table.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,no_lam,filled_lam,pa_Rural,pa_Semiurban,pa_Urban,pa_nan,bin_lam,bin_Low,bin_Average,bin_High,ch_bad,ch_good,ch_nan,apin_binned,apin_Low,apin_Average,apin_High,apin_nan,dep_0,dep_1,dep_2,dep_3+
0,Male,No,0,Graduate,No,5849,0.0,122.322222,360.0,1.0,Urban,1,1,146.412162,0,0,1,0,Low,1,0,0,0,1,0,Low,1,0,0,0,1,0,0,0
1,Male,Yes,1,Graduate,No,4583,1508.0,128.000000,360.0,1.0,Rural,0,0,128.000000,1,0,0,0,Low,1,0,0,0,1,0,Low,1,0,0,0,0,1,0,0
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.000000,360.0,1.0,Urban,1,0,66.000000,0,0,1,0,Low,1,0,0,0,1,0,Low,1,0,0,0,1,0,0,0
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.000000,360.0,1.0,Urban,1,0,120.000000,0,0,1,0,Low,1,0,0,0,1,0,Low,1,0,0,0,1,0,0,0
4,Male,No,0,Graduate,No,6000,0.0,141.000000,360.0,1.0,Urban,1,0,141.000000,0,0,1,0,Low,1,0,0,0,1,0,Low,1,0,0,0,1,0,0,0


<hr>
<h1>
You are done
</h1>
<p>
<div class=h1_cell>
<p>
Good job.
</div>

In [115]:
import os

week = 7  # change this each week

home_path =  os.path.expanduser('~')

file_name = 'loan_wrangled_w'+str(week)+'.csv'

file_path = '/Desktop/CIS399/'


loan_table.to_csv(home_path + file_path + file_name, index=False)